# Performance Run Analysis

In [1]:
# ---------------------------------------------------------------------------------------------
# MIT License
# Copyright (c) 2020, Solace Corporation, Jochen Traunecker (jochen.traunecker@solace.com)
# Copyright (c) 2020, Solace Corporation, Ricardo Gomez-Ulmke (ricardo.gomez-ulmke@solace.com)
# ---------------------------------------------------------------------------------------------

# Check if running in production mode 
# if not, set devel envs here

import os
from distutils.util import strtobool

notebook_run_production_mode_env_val=os.getenv("NOTEBOOK_RUN_PRODUCTION_MODE")
if notebook_run_production_mode_env_val is not None:
    is_notebook_run_production_mode = bool(strtobool(notebook_run_production_mode_env_val))
else:
    is_notebook_run_production_mode = False

if not is_notebook_run_production_mode:
    import sys, re
    analytics_path = re.sub(r'/analytics/.*$', '/analytics', sys.path[0])
    print("analytics_path={}".format(analytics_path))
    sys.path[0] = analytics_path + "/notebooks"
    print("sys.path[0]={}".format(sys.path[0]))

    os.environ['ANALYTICS_ENABLE_LOGGING'] = "true"    
    os.environ['ANALYTICS_LOG_FILE'] = "./logs/run-analysis.ipynb.log"    
    # azure
    # os.environ['NOTEBOOK_RESULTS_LOCATION_DIR'] = "../../uc-non-persistent/test-results/stats/azure.1-auto-standalone"
    # os.environ['NOTEBOOK_RESULTS_LOCATION_DIR'] = "../../uc-non-persistent/test-results/stats/azure.2-auto-standalone"
    os.environ['NOTEBOOK_RESULTS_LOCATION_DIR'] = "../../uc-non-persistent/test-results/stats/azure.devel1-standalone"
    # aws
    # os.environ['NOTEBOOK_RESULTS_LOCATION_DIR'] = "../../uc-non-persistent/test-results/stats/aws.1-auto-standalone"
    # os.environ['NOTEBOOK_RESULTS_LOCATION_DIR'] = "../../uc-non-persistent/test-results/stats/aws.devel1-standalone"

    os.environ['NOTEBOOK_RUN_ID'] = "2020-11-14-11-22-41"
    
    
if not is_notebook_run_production_mode:
    # print env vars
    print("ANALYTICS_ENABLE_LOGGING={}".format(os.getenv('ANALYTICS_ENABLE_LOGGING')))
    print("ANALYTICS_LOG_FILE={}".format(os.getenv('ANALYTICS_LOG_FILE')))
    print("NOTEBOOK_RESULTS_LOCATION_DIR={}".format(os.getenv('NOTEBOOK_RESULTS_LOCATION_DIR')))
    print("NOTEBOOK_RUN_ID={}".format(os.getenv('NOTEBOOK_RUN_ID')))


analytics_path=/Users/rjgu/Dropbox/Solace-Contents/Solace-IoT-Team/azure-performance/az-use-case-perf-tests/analytics
sys.path[0]=/Users/rjgu/Dropbox/Solace-Contents/Solace-IoT-Team/azure-performance/az-use-case-perf-tests/analytics/notebooks
ANALYTICS_ENABLE_LOGGING=true
ANALYTICS_LOG_FILE=./logs/run-analysis.ipynb.log
NOTEBOOK_RESULTS_LOCATION_DIR=../../uc-non-persistent/test-results/stats/azure.devel1-standalone
NOTEBOOK_RUN_ID=2020-11-14-11-22-41


In [2]:
from lib import run_definition
from lib import run_result_location
from lib import constants
from lib import run
from lib import run_analytics
from os import path

import json

# import numpy as np
# import pandas as pd
# from jsonpath_ng import jsonpath, parse
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

results_location_dir = os.getenv('NOTEBOOK_RESULTS_LOCATION_DIR')
run_id = os.getenv('NOTEBOOK_RUN_ID')

# import logging
# logging.debug("results_location_dir='{}'".format(results_location_dir))
# print("results_location_dir='{}'".format(results_location_dir))
# logging.debug("run_id='{}'".format(run_id))
# print("run_id='{}'".format(run_id))

# plt.rcParams["figure.figsize"] = [50, 50]

location = run_result_location.RunResultLocation(results_location_dir)

runDefinition = run_definition.RunDefinition(location)
runDefinition.process_run_samples()
run = runDefinition.find_run(run_id, True)
run_analytics = run_analytics.RunAnalytics(run)

if not is_notebook_run_production_mode:
    import logging
    logging.debug(f"run={run}")





In [3]:
from IPython.display import Markdown, display
display(Markdown(run.run_meta.getUseCaseAsMarkdown()))


## Use Case: Market Data Distribution (uc_non_persistent)

Test Specification: 
tp-001: artificial test for base line comparison.
(devel_tp_001)

Cloud Provider: **Azure**
        

In [4]:
from IPython.display import Markdown, display
display(Markdown(run_analytics.getChecksAsMarkdown()))


Checks: zero-message-loss:**<span style='color:green'>passing</span>** | message-tally:**<span style='color:green'>passing</span>**
        

In [5]:
if not run.success:
    import logging, sys
    logging.error(f"run was not successful - check error logs")
    sys.exit(0)

In [6]:
from IPython.display import Markdown, display
display(Markdown(run.broker_series.getOverviewAsMarkdown()))



## Run Metrics Summary

Description: base run: 1 publisher, 40 consumers, 40 topics, fan-out=1:1, 50K msgs/sec, 100 bytes

|Monitors        |Messages|Discarded|Rate (1/sec)|Fan Out|
|:---------------|:------:|:-------:|:-----------:|:-----:|
|broker received:| 641  | 0  | 8 |1|
|broker sent:    | 641  | 0  | 8 |1.00|

|Load(*)                                                             |Messages                               |Discarded                                       |Rate (1/sec)                             |Fan Out                     |
|:-------------------------------------------------------------------|:-------------------------------------:|:----------------------------------------------:|:---------------------------------------:|:--------------------------:|
|publishers (1)     |1,573 |0 |10  |1                           |
|consumers (10)       |1,573  |0  |10   |1.00    |

_(*)Note: Run metrics are taken in-flight, therefor they may not match exactly._
        
        

In [7]:

if run.run_meta.getRuncSpecLoadIsIncluded():
    import pandas as pd
    import plotly.express as px

    consumer_messages_receieved_df = run_analytics.export_consumer_messages_received_as_dataframe()
    bar_chart = px.bar(consumer_messages_receieved_df, \
                        title='Consumers: Messages Received', \
                        labels={'messages_received': '[number]', 'consumer_names': 'node:consumer-id'}, x='consumer_names', y='messages_received',\
                        height=300)
    bar_chart.show()


In [8]:
# Overview of Broker and Latency-Node Latencies
# To render both categories in one bar-plot it is best to use just standard Python data structures
# box_data is a dict with 'metrics':list(string): the metrics labels/names
#                         'latency-node':list(float) : the calculated metrics in the order of metric-labels
#                         'broker-node':list(float) : the calculated metrics in the order of metric-labels
#
bar_data = run_analytics.export_combined_all_distinct_latencies_metrics()

import plotly.graph_objects as go

title = "Latency Summary Stats: "
sep = ""
displayChart = False
if run.run_meta.getRuncSpecMonitorLatencyLatencyNodeIsIncluded():
    title += "Latency Node (Network)"
    sep = ", "
    displayChart = True
if run.run_meta.getRuncSpecMonitorLatencyBrokerNodeIsIncluded():
    title += sep + "Broker Node (local)"    
    displayChart = True

if displayChart:
    bar_fig = go.Figure(
        data =[
            go.Bar(
                name="Latency Node",
                x=bar_data['metrics'],
                y=bar_data['latency-node'],
                offsetgroup=0
            ),
            go.Bar(
                name="Broker Node",
                x=bar_data['metrics'],
                y=bar_data['broker-node'],
                offsetgroup=1
            )
        ],
        layout=go.Layout(
            title=title,
            yaxis_title="latency [micros]"
        )
    )
    bar_fig.show()
else:
    from IPython.display import Markdown, display
    md = f"""
### {title}
No latency monitors included.
    """
    display(Markdown(md))


In [9]:

from IPython.display import Markdown, display
display(Markdown(run.run_meta.getAsMarkdown()))




## Run Settings
* Description: "base run: 1 publisher, 40 consumers, 40 topics, fan-out=1:1, 50K msgs/sec, 100 bytes"

|General                    |                                                   | | Infrastructure:           | cloud provider:azure                                        |      |   |  
|:--------------------------|:--------------------------------------------------|-|:--------------------------|:----------------------------------------------------------------------------|:-----|:--|
|Run name:                  |base_run                                    | |azure.devel1-standalone      |region: West Europe, zone: 1                     |      |   |                                              
|Run Id:                    |2020-11-14-11-22-41                                      | |Broker Node:               |nodes: 1<br/>size: Standard_F16s_v2             |      |   |
|Run Start:                 |2020-11-14 11:23:57                                | |Load<br/>Publisher Nodes:  |nodes: 1<br/>size: Standard_F4s_v2       |      |   |
|Run End:                   |2020-11-14 11:26:53                                  | |Load<br/>Consumer Nodes:   |nodes: 4 <br/>size: Standard_F4s_v2        |      |   |
|Run Duration:              |0:02:56                              | |Monitor Node:              |nodes: 1 <br/>size: Standard_F4s_v2          |      |   |  
|Sample Duration (secs):    |60        | |Solace PubSub+             | solace-pubsub-evaluation:9.7.0.29                                                |      |   | 
|Number of Samples:         |2           | |                           |                                                                             |      |   | 


|Load|                                                                          | | Monitors   |                              |                                                                 |  
|:--|:--------------------------------------------------------------------------|-|:-----------|:-----------------------------|:----------------------------------------------------------------|
|Included:              |**True**                 | |**Latency** |                              |                                                                 |  
|Connections @ start:   |11                | | |Latency Node to Broker Node - included:  |**True**      |    
|Connections @ end:     |11                  | | |Broker Node to Broker Node - included:   |**True**       |      
|Publishers:            |1              | | |Payload (bytes):                         |100             |      
| - Payload (bytes):    |100      | | |Rate (1/sec):                            |1                   |      
| - Rate (1/sec):       |10       | |**Ping**       | included:                 |**True**                    |  
| - Topics:             |10      | |**Broker VPN** | included:                 |**True**                     |  
|Consumers:             |10 | |               |                           |                                                                 |   
| - deploy strategy:    |round_robin   | |               |                           |                                                                 |   

            

## Latency Stats: Latency Node to Broker Node (Network)

In [10]:
if run.run_meta.getRuncSpecMonitorLatencyLatencyNodeIsIncluded():
    import plotly.express as px
    df_lat_lat_node = run_analytics.export_latency_node_series_latencies_metrics_as_dataframe()
    fig_line = px.line(df_lat_lat_node,
                    title="Latency Percentiles: Latency Node to Broker Node (Network)",
                    labels={"value":"latency [micros]", "index":"sample [number]"})
    fig_line.show()
else: 
    print("Latency Node to Broker Node: not included")


## Latency Stats: Broker Node to Broker Node (local)

In [11]:
if run.run_meta.getRuncSpecMonitorLatencyBrokerNodeIsIncluded():
    import plotly.express as px
    df_lat_broker_node = run_analytics.export_broker_node_series_latencies_metrics_as_dataframe()
    fig_line = px.line(df_lat_broker_node,
                    title="Latency Percentiles: Broker Node to Broker Node (local)",
                    labels={"value":"latency [micros]", "index":"sample [number]"})
    fig_line.show()
else: 
    print("Broker Node to Broker Node: not included")

## Latency (raw data): Latency Node to Broker Node (Network)

In [12]:
if run.run_meta.getRuncSpecMonitorLatencyLatencyNodeIsIncluded():
    import plotly.express as px
    df_lat_raw_lat_node = run_analytics.export_latency_node_distinct_latencies_as_dataframe()
    fig_line = px.line(df_lat_raw_lat_node,
                    title="Latency (raw data): Latency Node to Broker Node (network)",
                    labels={"value":"latency [micros]", "index":"message [number]"})
    fig_line.show()
else: 
    print("Latency Node to Broker Node: not included")

## Latency (raw data): Broker Node to Broker Node (local)

In [13]:
if run.run_meta.getRuncSpecMonitorLatencyBrokerNodeIsIncluded():
    import plotly.express as px
    df_lat_raw_broker_node = run_analytics.export_broker_node_distinct_latencies_as_dataframe()
    fig_line = px.line(df_lat_raw_broker_node,
                    title="Latency (raw data): Broker Node to Broker Node (local)",
                    labels={"value":"latency [micros]", "index":"message [number]"})
    fig_line.show()
else: 
    print("Broker Node to Broker Node: not included")

## Ping Statistics: Latency Node to Broker Node (Network)

In [14]:
if run.run_meta.getRuncSpecMonitorPingIsIncluded():
    import plotly.express as px
    df_ping = run_analytics.export_ping_series_as_columns_dataframe()
    fig_line = px.line(df_ping,
                        title="Ping Statistics: Latency Node to Broker Node (Network)",
                        labels={"value":"latency [micros]", "index":"sample [number]"})
    fig_line.show()    
else: 
    print("Ping Statistics: not included")

---
The End.